In [92]:
#!pip install torch torchtext sentencepiece datasets
# Try opus books dataset for translation - https://huggingface.co/datasets/opus_books

In [1]:
import numpy as np
import torch
from torch import nn
import functorch
import sys
import os
import math
sys.path.append(os.path.abspath("../../data"))

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
BATCH_SIZE = 32
SP_VOCAB_SIZE = 1000
VOCAB_SIZE = SP_VOCAB_SIZE + 2
CHUNK_LENGTH = 15
Y_CHUNK_LENGTH = 15

/Users/vik/.virtualenvs/nnets/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from cnn_data import generate_data, decode_batch

train, valid, test = generate_data(train_length=1000, valid_length=250, vocab_size=SP_VOCAB_SIZE, batch_size=BATCH_SIZE, chunk_length=CHUNK_LENGTH, y_chunk_length=Y_CHUNK_LENGTH, device=DEVICE)

Found cached dataset cnn_dailymail (/Users/vik/.cache/huggingface/datasets/cnn_dailymail/3.0.0/3.0.0/1b3c71476f6d152c31c1730e83ccb08bcf23e348233f4fcc11e182248e6bf7de)
100%|██████████| 3/3 [00:00<00:00, 153.97it/s]
sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=tokens.txt --model_prefix=cnn --vocab_size=1000 --model_type=unigram
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: tokens.txt
  input_format: 
  model_prefix: cnn
  model_type: UNIGRAM
  vocab_size: 1000
  self_test_sample_size: 0
  character_coverage: 0.9995
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  treat_whitespace_as_suffix: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_output_piece_score: 1
  train_

In [58]:
class GRUEncoderCell(nn.Module):
    def __init__(self, input_units, hidden_units, output_units):
        super(GRUEncoderCell, self).__init__()
        self.input_units = input_units
        self.hidden_units = hidden_units
        self.output_units = output_units

        k = 1e-2
        self.input_weights = nn.Parameter(torch.rand(3, input_units, hidden_units) * 2 * k - k)

        self.hidden_weights = nn.Parameter(torch.rand(3, hidden_units, hidden_units) * 2 * k - k)
        self.hidden_biases = nn.Parameter(torch.rand(2, 1, hidden_units) * 2 * k - k)

        self.output_weight = nn.Parameter(torch.rand(hidden_units, output_units) * 2 * k - k)
        self.output_bias = nn.Parameter(torch.rand(1, output_units) * 2 * k - k)

    def forward(self, x, prev_hidden):
        # Compute the regular RNN forward pass
        # Compute update and reset gates for GRU
        update_gate = torch.sigmoid(x @ self.input_weights[0,] + prev_hidden @ self.hidden_weights[0,] + self.hidden_biases[0,])
        reset_gate = torch.sigmoid(x @ self.input_weights[1,] + prev_hidden @ self.hidden_weights[1,] + self.hidden_biases[1,])

        # This is a potential new state based on the reset gate
        proposed_state = torch.tanh(x @ self.input_weights[2,] + (prev_hidden * reset_gate) @ self.hidden_weights[2,])
        hidden_x = torch.tanh((1-update_gate) * prev_hidden + update_gate * proposed_state)
        # Compute output vector
        output_y = hidden_x @ self.output_weight + self.output_bias
        return output_y, hidden_x

In [71]:
class GRUDecoderCell(nn.Module):
    def __init__(self, input_units, context_units, hidden_units, output_units):
        super(GRUDecoderCell, self).__init__()
        self.input_units = input_units
        self.hidden_units = hidden_units
        self.output_units = output_units

        k = 1e-2

        self.input_weights = nn.Parameter(torch.rand(3, input_units, hidden_units) * 2 * k - k)
        self.hidden_weights = nn.Parameter(torch.rand(3, hidden_units, hidden_units) * 2 * k - k)

        self.context_weights = nn.Parameter(torch.rand(3, context_units, hidden_units) * 2 * k - k)

        self.hidden_biases = nn.Parameter(torch.rand(2, 1, hidden_units) * 2 * k - k)

        self.hidden_attention_weight = nn.Parameter(torch.rand(hidden_units, hidden_units) * 2 * k - k)
        self.context_attention_weight = nn.Parameter(torch.rand(context_units, hidden_units) * 2 * k - k)
        self.attention_weight = nn.Parameter(torch.rand(1, hidden_units) * 2 * k - k)

        self.output_weight = nn.Parameter(torch.rand(hidden_units, output_units) * 2 * k - k)
        self.output_bias = nn.Parameter(torch.rand(1, output_units) * 2 * k - k)

        self.batched_diag = functorch.vmap(torch.diag)

    def forward(self, prev_y, prev_hidden, context):
        # Compute attention between the encoder hidden states and the previous decoder hidden state

        # Swap batch and sequence
        batch_size = context.shape[1]

        # Swap axes so the first dimension of context_attn is batch
        context_attn = torch.bmm(context.swapaxes(0,1), self.context_attention_weight.unsqueeze(0).expand(batch_size,-1,-1))
        # Swap back since prev_hidden is by batch.  This makes the first dim of cross sequence
        cross = torch.tanh(context_attn.swapaxes(0,1) + prev_hidden @ self.hidden_attention_weight)
        # This will be of dimension batch, sequence_length, 1
        attention = torch.bmm(cross.swapaxes(0,1), self.attention_weight.T.unsqueeze(0).expand(batch_size, -1, -1))
        # Drop the last singleton dimension
        attention = attention.squeeze(2)
        # Softmax the predictions across each batch
        probs = torch.softmax(attention, 1)
        diagonalized_probs = self.batched_diag(probs)
        positional_contexts = torch.sum(torch.bmm(diagonalized_probs, context.swapaxes(0,1)), dim=1).reshape(batch_size, self.input_units)

        # Compute GRU update and reset gates + proposed state and final hidden state
        update_gate = torch.sigmoid(prev_y @ self.input_weights[0,] + prev_hidden @ self.hidden_weights[0,] + self.hidden_biases[0,] + positional_contexts @ self.context_weights[0,])
        reset_gate = torch.sigmoid(prev_y @ self.input_weights[1,] + prev_hidden @ self.hidden_weights[1,] + self.hidden_biases[1,] + positional_contexts @ self.context_weights[1,])

        proposed_state = torch.tanh(prev_y @ self.input_weights[2,] + (prev_hidden * reset_gate) @ self.hidden_weights[2,] + positional_contexts @ self.context_weights[2,])

        hidden_x = torch.tanh((1-update_gate) * prev_hidden + update_gate * proposed_state)
        # Compute output based on hidden state
        output_y = hidden_x @ self.output_weight + self.output_bias
        return output_y, hidden_x

In [72]:
class EncoderDecoder(nn.Module):
    def __init__(self, in_sequence_len, out_sequence_len, hidden_units=512, embedding_len=VOCAB_SIZE, layers=2):
        super(EncoderDecoder, self).__init__()
        self.in_sequence_len = in_sequence_len
        self.out_sequence_len = out_sequence_len
        self.hidden_units = hidden_units
        self.embedding_len = embedding_len
        self.layers = layers

        self.embedding = nn.Embedding(embedding_len, hidden_units)
        self.encoders = nn.ModuleList([GRUEncoderCell(input_units=hidden_units, hidden_units=hidden_units, output_units=hidden_units) for _ in range(layers)])
        self.decoders = nn.ModuleList([GRUDecoderCell(input_units=hidden_units, context_units=hidden_units, hidden_units=hidden_units, output_units=hidden_units) for _ in range(layers)])

        self.linear = nn.Linear(in_features=hidden_units, out_features=embedding_len)

    def forward(self, x, y):
        batch_size = x.shape[0]
        # Move batch to the second dimension, so sequence comes first
        y = y.swapaxes(0,1)
        # Embed the input sequence to reduce dimensionality
        embedded = self.embedding(x).swapaxes(0,1)

        # Encode the input sequence
        # Both tensors will have sequence then batch
        enc_hiddens = torch.zeros((1, self.layers, batch_size, self.hidden_units), device=DEVICE)
        enc_outputs = torch.zeros((1, self.layers, batch_size, self.hidden_units), device=DEVICE)
        for j in range(self.in_sequence_len):
            seq_enc_hiddens = torch.zeros((1, batch_size, self.hidden_units), device=DEVICE)
            seq_enc_outputs = embedded[j,:,:].unsqueeze(0).to(DEVICE)
            for i in range(self.layers):
                output, hidden = self.encoders[i](seq_enc_outputs[i,], enc_hiddens[j,i])
                # Add first sequence axis
                hidden = hidden.unsqueeze(0)
                output = output.unsqueeze(0)
                seq_enc_outputs = torch.cat((seq_enc_outputs, output), dim=0)
                seq_enc_hiddens = torch.cat((seq_enc_hiddens, hidden), dim=0)

            enc_hiddens = torch.cat((enc_hiddens, seq_enc_hiddens[1:].unsqueeze(0)), dim=0)
            enc_outputs = torch.cat((enc_outputs, seq_enc_outputs[1:].unsqueeze(0)), dim=0)

        # Decode to the output sequence
        # Pass in context
        context = enc_hiddens[1:,-1,:,:]
        # Both tensors will have the first dimension be the sequence
        dec_hiddens = torch.zeros(1, self.layers, batch_size, self.hidden_units, device=DEVICE)
        dec_outputs = torch.zeros((1, self.layers, batch_size, self.hidden_units), device=DEVICE)
        for j in range(self.out_sequence_len):
            # Use either the actual previous y (from the input), or the generated y if the input sequence is shorter than the generation steps.
            prev_y = y[j,:,:] if y.shape[0] > j else torch.softmax(dec_outputs[j,-1,:,:], dim=1)
            # Run embedding over previous y state
            prev_y = prev_y.argmax(dim=1).int()
            prev_y = self.embedding(prev_y)

            seq_dec_hiddens = torch.zeros((1, batch_size, self.hidden_units), device=DEVICE)
            seq_dec_outputs = prev_y.unsqueeze(0).to(DEVICE)
            for i in range(self.layers):
                output, hidden = self.decoders[i](seq_dec_outputs[i,], dec_hiddens[j,i,], context)
                # Add first sequence axis
                hidden = hidden.unsqueeze(0)
                output = output.unsqueeze(0)
                seq_dec_outputs = torch.cat((seq_dec_outputs, output), dim=0)
                seq_dec_hiddens = torch.cat((seq_dec_hiddens, hidden), dim=0)

            dec_hiddens = torch.cat((dec_hiddens, seq_dec_hiddens[1:].unsqueeze(0)), dim=0)
            dec_outputs = torch.cat((dec_outputs, seq_dec_outputs[1:].unsqueeze(0)), dim=0)

        # Move batch back to axis 0
        out_hiddens = dec_hiddens[1:,-1,:,:].swapaxes(0,1)
        out_output = self.linear(dec_outputs[1:,-1,:,:].swapaxes(0,1))
        return out_output, out_hiddens

def generate(sequence, target, vocab_size):
    pred, _ = model(sequence, target[:,0,:].unsqueeze(1))
    prompts = decode_batch(sequence.cpu(), vocab_size=vocab_size)
    texts = decode_batch(torch.argmax(pred, dim=2).cpu(), vocab_size=vocab_size)
    correct_texts = decode_batch(torch.argmax(target, dim=2).cpu(), vocab_size=vocab_size)

    displays = []
    for p, t, ct in zip(prompts, texts, correct_texts):
        displays.append(f"{p} | {ct} | {t}")
    return displays

In [75]:
from tqdm.auto import tqdm
model = EncoderDecoder(CHUNK_LENGTH, Y_CHUNK_LENGTH, hidden_units=512, layers=1).to(DEVICE)
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)

In [ ]:
EPOCHS = 2000
DISPLAY_BATCHES = 8

for epoch in range(EPOCHS):
    # Run over the training examples
    train_loss = 0
    for batch, (sequence, target, prev_target) in tqdm(enumerate(train)):
        optimizer.zero_grad()
        forced_target = prev_target
        # Alternate use of teacher forcing vs feeding back own inputs
        if np.random.randint(2) == 0:
            forced_target = prev_target[:,0,:].unsqueeze(1)
        pred, hidden = model(sequence, forced_target)
        loss = loss_fn(pred, target)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    with torch.no_grad():
        if epoch % 10 == 0:
            # Show text generated from training prompt as an example
            # Don't feed in all of the train y sequences, just the first token
            # The other y tokens will be predicted by the model and fed back in
            sents = generate(sequence[:DISPLAY_BATCHES], prev_target[:DISPLAY_BATCHES], SP_VOCAB_SIZE)
            for sent in sents:
                print(sent)

            # Compute validation loss.  Unless you have a lot of training data, the validation loss won't decrease.
            valid_loss = 0
            for batch, (sequence, target, prev_target) in enumerate(valid):
                # Only feed in the first token of the actual target
                pred, hidden = model(sequence, prev_target[:,0,:].unsqueeze(1))
                loss = loss_fn(pred, target)
                valid_loss += loss.item()
            print(f"Epoch {epoch} train loss: {train_loss} valid loss: {valid_loss}")

32it [00:07,  4.03it/s]


NEW: Georgian president criticizes Gorbachev for " | NEW: Georgian president criticizes Gorbachev for | TheEWEWEWEWEW, with with with with with with with with
19 schoolgirls and two adults die in | 19 schoolgirls and two adults die | TheEWEW:EWEWEWEWEWEWEWEWEWEWEW
U.S. intelligence points to Pakistan | U.S. intelligence points to | TheEWEW: New TwEWeeEWEWee Thes
Mother of murdered schoolboy Damilola Tay | Mother of murdered schoolboy Damilola T | TheEWEW: New TwEWEWEWEWEWEWEWEWEW
NEW: Pope Benedict  ⁇ VI | NEW: Pope Benedict  ⁇ V | TheEWEWEWEWEW, with with with with with with with with
Eight Florida teens to be tried as adults | Eight Florida teens to be tried as adult | TheEWEW:t TwEWEWEWEWEWEW Thess
Judge on Heather Mills: Level of | Judge on Heather Mills: Level | TheEWEW:: TwEWEWEWEWEWEWEWEWEW
President Harding's illegitimate daughter | President Harding's illegitimate | TheEWEW:EWEWEWEWEWEWEWEWEWEWEW
Epoch 0 train loss: 1.2102768123149872 valid loss: 0.3133130967617035


32it [00:07,  4.01it/s]
32it [00:07,  4.12it/s]
32it [00:07,  4.03it/s]
32it [00:07,  4.05it/s]
32it [00:07,  4.05it/s]
32it [00:07,  4.02it/s]
32it [00:07,  4.02it/s]
32it [00:08,  3.99it/s]
32it [00:08,  3.93it/s]
32it [00:08,  3.90it/s]


NEW: Georgian president criticizes Gorbachev for " | NEW: Georgian president criticizes Gorbachev for | FormEW Police British Department Department Department a a a a a a a a
19 schoolgirls and two adults die in | 19 schoolgirls and two adults die | TheEWReportology members'''''' un un un un
U.S. intelligence points to Pakistan | U.S. intelligence points to | TheEWN Department re re'''' un un un un un
Mother of murdered schoolboy Damilola Tay | Mother of murdered schoolboy Damilola T | TheEW force British,000,000,000,000,000,000,000,000,000,000,000
NEW: Pope Benedict  ⁇ VI | NEW: Pope Benedict  ⁇ V | FormEW Police British Department members members members members members a a a a a
Eight Florida teens to be tried as adults | Eight Florida teens to be tried as adult | TheEW countriesology''''' attempt attempt attempt un attempt attempt
Judge on Heather Mills: Level of | Judge on Heather Mills: Level | TheEW force British leader leader,000,000 woman woman''' Mor Mor
President Harding's i

32it [00:08,  3.92it/s]
32it [00:08,  3.87it/s]
32it [00:08,  3.86it/s]
32it [00:08,  3.94it/s]
32it [00:08,  3.96it/s]
32it [00:08,  3.87it/s]
32it [00:08,  4.00it/s]
32it [00:08,  3.96it/s]
32it [00:07,  4.06it/s]
32it [00:07,  4.04it/s]


NEW: Georgian president criticizes Gorbachev for " | NEW: Georgian president criticizes Gorbachev for | FormEW British British video video,000tontontontontontontonton
19 schoolgirls and two adults die in | 19 schoolgirls and two adults die | FBIEWN receive NATOhip mark custody custody11111 alleged
U.S. intelligence points to Pakistan | U.S. intelligence points to | FBI British children memberstonton bomb bomb bomb bomb bomb arm arm arm arm
Mother of murdered schoolboy Damilola Tay | Mother of murdered schoolboy Damilola T | FBI Bush British British,000,000tontontontontontontontonton
NEW: Pope Benedict  ⁇ VI | NEW: Pope Benedict  ⁇ V | FormEW British British video videotontontontontontontontonton
Eight Florida teens to be tried as adults | Eight Florida teens to be tried as adult | FBIEW countries McCain NATO NATO NATO NATO6 re re re re re re
Judge on Heather Mills: Level of | Judge on Heather Mills: Level | FBIH week British British British British British British British McCain McCain

32it [00:07,  4.03it/s]
32it [00:08,  3.93it/s]
32it [00:08,  3.95it/s]
32it [00:08,  3.98it/s]
32it [00:07,  4.01it/s]
32it [00:07,  4.08it/s]
32it [00:08,  3.97it/s]
32it [00:08,  3.90it/s]
10it [00:02,  3.98it/s]